In [3]:
import cv2
from djitellopy import Tello
from ultralytics import YOLO

# model = YOLO('best_smoke.pt')

In [ ]:
def search_and_return(tello: Tello, target_class_id=0, search_duration=10):
    '''
    在目标丢失后，Tello无人机通过悬停旋转寻找目标。
    如果找不到，则返回初始方向。
    
    :param tello: Tello实例
    :param target_class_id: 需要识别的目标类别ID，默认为0
    :param search_duration: 搜索持续时间（秒）
    '''
    initial_attitude = tello.get_attitude()
    initial_yaw = initial_attitude.yaw  # 记录初始朝向
    
    # 计算总共需要旋转的次数以达到每秒半圈，这里假设每次旋转180度
    rotations_per_second = 0.5  # 半圈
    total_rotations = rotations_per_second * search_duration
    degrees_per_rotation = 60  # 假设每次旋转60度

    current_yaw = initial_yaw  # 用于追踪当前的偏航角

    for _ in range(int(total_rotations)):
        # 旋转无人机
        tello.rotate_clockwise(degrees_per_rotation / 2)  # 每次旋转30度
        frame_read = tello.get_frame_read().frame
        results = model(frame_read, classes=[target_class_id], verbose=False)

        # 绘制检测结果（可选）
        annotated_frame = results[0].plot() if results else frame_read.copy()  # 确保无结果时仍显示原始帧
        cv2.imshow('Drone Camera', annotated_frame)
        cv2.waitKey(1)  # 显示视频帧，非阻塞模式

        if results and len(results[0].boxes.data):  # 如果找到了目标
            print('目标已找到！')
            break
        else:
            current_yaw += degrees_per_rotation / 2  # 更新当前偏航角

    # 计算并执行返回初始方向的旋转
    tello.rotate_counter_clockwise((initial_yaw - current_yaw + 360) % 360)  # 处理角度循环问题
    print('已返回初始方向')